In [2]:
import numpy as np, numpy.random
import pandas as pd
from sklearn.preprocessing import normalize
import copy

Import Dataset

In [3]:
from google.colab import files
data_to_load = files.upload()

Saving final.csv to final.csv


In [4]:
import io
truth_data = pd.read_csv(io.BytesIO(data_to_load['final.csv']))
truth_data = truth_data.loc[:, ['TTL', 'STDEV','Pos']]
truth_data = truth_data.assign(Player_ID=range(len(truth_data)))
truth_data = truth_data.rename(columns={"TTL": "True Value", "STDEV": "STD",'Pos':'Position','Player_ID':'Player ID'})

In [5]:
constraints = {"QB":1, "RB":2, "WR": 2, "TE": 1, "K":1, "DST":1}
player_dict = {}
for i in constraints.keys():
  player_dict[i] = list(truth_data.loc[truth_data['Position'] == i]["Player ID"])

In [6]:
def generate_noisy_values(N,truth_data):
  values = np.ones((N,len(truth_data)))
  for n in range(N):
    for player_id in range(len(truth_data)):
      value = np.random.normal(pd.to_numeric(truth_data["True Value"][player_id]), pd.to_numeric(truth_data["STD"][player_id]), 1)[0]
      if(value<0):
        value = 0
      values[n,player_id] = value
  values = normalize(values, axis=1, norm='l1')
  return values

In [7]:
def draft(values, players_dict, constraints,N,naive):
  V = np.copy(values)
  players_left = copy.deepcopy(players_dict);
  allocation_dict = {}
  for n in range(N):
    allocation_dict[n]=[]
  order = list(range(N))
  np.random.shuffle(order)
  contin = True
  #continue until all players are allocated
  while(contin):
    agents_full_status = numpy.full((N, len(players_left.keys())), False)
    for n in order:
      opportunity_cost = []
      best_option = []
      pools = list(players_left.keys())
      #for each person, they need to figure out which pool to pick the best player for. To do so, we'll look at the opportunity cost
      for i in range(len(pools)):
        pool = pools[i]
        num_already_in_class = len(set(allocation_dict[n]).intersection(players_dict[pool]))
        num_needed = constraints[pool] - num_already_in_class
        #if that class is full, no need to consider it
        if(num_needed <= 0):
          agents_full_status[n,i] = True
          opportunity_cost.append(-1)
          best_option.append(-1)
        else:
          #figure out how many turns before they can go again in this class
          risky_agents = [j for j in range(n,N) if agents_full_status[j,i]==False]
          turns = len(risky_agents)
          #check if they need more than 1 player from that pool to see if they can potentially pick 2 before n picks again
          for comp in risky_agents:
            comp_num = constraints[pool] - len(set(allocation_dict[comp]).intersection(players_dict[pool]))
            if(comp_num>1):
              turns = turns + 1
          #calculate value of remaining players
          pool_values = []
          for player in players_left[pool]:
            pool_values.append(V[n,player])
          #if you are guaranteed the last few you're good
          if(num_needed == len(players_left[pool])):
            opportunity_cost.append(0)
          else:
            #calculate opportunity_cost
            if (turns>=len(players_left[pool])):
              opportunity_cost.append(max(pool_values) - min(pool_values))
            else:
              opportunity_cost.append(max(pool_values) - sorted(pool_values)[-turns])
          best_option.append(players_left[pool][pool_values.index(max(pool_values))])
      #done with everything!
      if(max(opportunity_cost)==-1):
        agents_full_status[n,:] = True
      else:
        #if its naive then act greedily
        if(n in naive):
          values_of_remaining = []
          indices = []
          #compile the remaining player values
          for pool_i in range(len(players_left.keys())):
            if(opportunity_cost[pool_i]!=-1):
              for player in players_left[pools[pool_i]]:
                values_of_remaining.append(V[n,player])
                indices.append(player)
          #get greedy best and allocate
          greedy_best = indices[values_of_remaining.index(max(values_of_remaining))]
          chosen_pool = [i for i in players_dict if players_dict[i].count(greedy_best)>0][0]
          temp = allocation_dict[n]
          temp.append(greedy_best)
          allocation_dict[n] = temp
          #remove it from players_left
          temp_2 = players_left[chosen_pool]
          temp_2.remove(greedy_best)
          players_left[chosen_pool] = temp_2
          if(len(allocation_dict[n])>15):
            print(opportunity_cost)
        else:
          #get the pool with the highest opportunity cost
          index_of_pool = opportunity_cost.index(max(opportunity_cost))
          pool_name = list(constraints.keys())[index_of_pool]
          #allocate the player to n
          temp = allocation_dict[n]
          temp.append(best_option[index_of_pool])
          allocation_dict[n] = temp
          #remove it from players_left
          temp_2 = players_left[pool_name]
          temp_2.remove(best_option[index_of_pool])
          players_left[pool_name] = temp_2
    #all agents teams full
    if(agents_full_status.all()):
      contin = False
    #reverse for snake draft
    order.reverse()
  return allocation_dict;

In [8]:
def evaluate_allocation(values, allocation,N,truth_data):
  sw = 0
  norm_alloc = 0
  vals = []
  for n in range(N):
    alloc_value = 0
    for i in allocation[n]:
      alloc_value = alloc_value + values[n,i]
    vals.append(alloc_value*len(truth_data)/len(allocation[n]))
    norm_alloc = norm_alloc + alloc_value*len(truth_data)/len(allocation[n])
    sw = sw + alloc_value
  return sw, norm_alloc,vals

In [14]:
def run(N,truth_data,constraints,player_dict,naive):
  data = []
  for i in range(30):
    values = generate_noisy_values(N,truth_data)
    A = draft(values, player_dict, constraints,N,naive)
    sw, norm_alloc, vals = evaluate_allocation(values, A, N,truth_data)
    data.append(vals)
  from statistics import mean
  print(*map(mean, zip(*data)))

In [15]:
run(4, truth_data, constraints, player_dict,[])

2.576178100737645 2.5619443749545523 2.5393950016622218 2.511363308360277
